In [1]:
from scapy.all import rdpcap, TCP, IP

# Load the pcap file
pcap_file = "8.pcap"
packets = rdpcap(pcap_file)

# Variables to store results
matching_ips_1 = []  # For absolute difference in ports & ACK flag
matching_ips_2 = []  # For checksum and sequence number
port_sum_count = 0  # Count of packets where sum of ports is in range
ack_number_matching = []  # Packets with ack_number in range

# Given conditions
port_diff_target = 54286
ack_last_digits = "1203"
checksum_prefix = "b5"
seq_last_digits = "6183"
ack_min, ack_max = 1678999000, 1700000000
port_sum_min, port_sum_max = 10000, 20000

# Process packets
for pkt in packets:
    if pkt.haslayer(TCP) and pkt.haslayer(IP):
        tcp_layer = pkt[TCP]
        ip_layer = pkt[IP]
        src_port = tcp_layer.sport
        dst_port = tcp_layer.dport
        port_diff = abs(src_port - dst_port)
        port_sum = src_port + dst_port

        # Check for first condition (port difference and ACK flag)
        if port_diff == port_diff_target and tcp_layer.flags & 0x10:  # ACK flag
            if str(tcp_layer.ack)[-4:] == ack_last_digits:
                matching_ips_1.append((ip_layer.src, ip_layer.dst))

        # Check for second condition (checksum, seq number)
        if tcp_layer.chksum and hex(tcp_layer.chksum)[2:].startswith(checksum_prefix):
            if str(tcp_layer.seq)[-4:] == seq_last_digits:
                matching_ips_2.append((ip_layer.src, ip_layer.dst, hex(tcp_layer.chksum)))

        # Check for third condition (sum of ports in range)
        if port_sum_min <= port_sum <= port_sum_max:
            port_sum_count += 1

        # Check for fourth condition (ack number in range)
        if ack_min <= tcp_layer.ack <= ack_max:
            ack_number_matching.append((ip_layer.src, ip_layer.dst, tcp_layer.ack))

# Output results
print(f"Matching IPs for port difference & ACK flag: {matching_ips_1}")
print(f"Matching IPs for checksum & sequence number: {matching_ips_2}")
print(f"Count of packets with sum of ports in range: {port_sum_count}")
print(f"Packets with ACK number in range: {ack_number_matching}")


Matching IPs for port difference & ACK flag: [('10.7.11.235', '180.149.59.137'), ('10.7.11.235', '180.149.59.137')]
Matching IPs for checksum & sequence number: [('23.54.155.211', '10.7.11.235', '0xb55a')]
Count of packets with sum of ports in range: 0
Packets with ACK number in range: [('10.7.11.235', '104.18.32.47', 1691529412), ('10.7.11.235', '104.18.32.47', 1691529412), ('10.7.11.235', '104.18.32.47', 1691529412), ('10.7.11.235', '104.18.32.47', 1691530787), ('10.7.11.235', '104.18.32.47', 1691530787), ('10.7.11.235', '104.18.32.47', 1691530787), ('10.7.11.235', '104.18.32.47', 1691530787), ('10.7.11.235', '104.18.32.47', 1691530787), ('10.7.11.235', '104.18.32.47', 1691530787), ('10.7.11.235', '104.18.32.47', 1691530787), ('10.7.11.235', '104.18.32.47', 1691530787), ('10.7.11.235', '104.18.32.47', 1691530787), ('10.7.11.235', '104.18.32.47', 1691530787), ('10.7.11.235', '104.18.32.47', 1691530787), ('10.7.11.235', '104.18.32.47', 1691530787), ('10.7.11.235', '104.18.32.47', 16915